In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
import pandas as pd
from konlpy.tag import Hannanum
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [2]:
hannanum = Hannanum()

In [3]:
df_stat = pd.read_excel('dongduk_course_statistic.xlsx')
df_stat['학수번호'] = df_stat['학수번호'].str.strip() #학수번호 공백없애기 
df_stat = df_stat[['학수번호','과목소개']] 
df_stat.head()

,학수번호,과목소개
0,데이 K0040,인터넷프로그래밍 (Internet Programming) 가. 네트워크...
1,데이 K0047,선형모형입문 (Introduction to Linear Models) 가....
2,데이 K0056,정보시스템 보안 및 관리 (Information System Security...
3,데이 K0045,정보시스템활용 (Application of Information System...
4,데이 K0052,생명과학자료분석 (Data Analysis for the Life Scien...


In [4]:
df_manage = pd.read_excel('dongduk_course_manage.xlsx')
df_manage['학수번호'] = df_manage['학수번호'].str.strip() #학수번호 공백없애기 
df_manage = df_manage[['학수번호','과목소개']] 
df_manage.head()

,학수번호,과목소개
0,경영K0033,중급회계Ⅱ 기업에 대한 정보를 외부의 이해관계자들에게 생산 전달하는 재무...
1,경영B0001,경영학원론 경영학이론의 발전과정 경영학의 체계 현대기업의 주요 경영이론 ...
2,경영B0001,경영학원론 경영학이론의 발전과정 경영학의 체계 현대기업의 주요 경영이론 ...
3,경영K0037,서비스마케팅 산업화가 진전됨에 따라 산업구조도 제조업에서 서비스업 중심으...
4,경영K0009,광고론 광고이론을 체계적으로 이해시키며 타 마케팅관련 변수들과의 유기적 ...


In [5]:
df_stat = df_stat.append(df_manage)
df_stat = df_stat.reset_index(drop=True)
df_stat.tail()

,학수번호,과목소개
67,경영K0013,세법개론 본 과목은 기업의 경영과 개인의 일상생활에서 중요성이 증대되고 ...
68,경영K0034,고급재무회계 중급회계에서 다루지 않았던 재무회계의 특수 분야인 고급수준의...
69,경영K0003,조직행동론 조직 내에서의 인간행동에 대한 원인을 규명하여 조직의 효율성을...
70,경영K0019,세무회계 본 과목은 세무회계의 이론과 실무를 이해시키고 기업회계와 세법의...
71,경영K0017,경영정보시스템 조직운영 및 경영활동을 지원하는 경영정보시스템에 관련된 학...


In [6]:
df_stat

,학수번호,과목소개
0,데이 K0040,인터넷프로그래밍 (Internet Programming) 가. 네트워크...
1,데이 K0047,선형모형입문 (Introduction to Linear Models) 가....
2,데이 K0056,정보시스템 보안 및 관리 (Information System Security...
3,데이 K0045,정보시스템활용 (Application of Information System...
4,데이 K0052,생명과학자료분석 (Data Analysis for the Life Scien...
...,...,...
67,경영K0013,세법개론 본 과목은 기업의 경영과 개인의 일상생활에서 중요성이 증대되고 ...
68,경영K0034,고급재무회계 중급회계에서 다루지 않았던 재무회계의 특수 분야인 고급수준의...
69,경영K0003,조직행동론 조직 내에서의 인간행동에 대한 원인을 규명하여 조직의 효율성을...
70,경영K0019,세무회계 본 과목은 세무회계의 이론과 실무를 이해시키고 기업회계와 세법의...


In [51]:
df_stat.to_csv('manage+stat.csv',index=False)

In [7]:
docs = []
for i in df_stat['과목소개']:  
    docs.append(hannanum.nouns(i))
    
for i in range(len(docs)):
    docs[i] = ' '.join(docs[i])
df_stat['과목소개'] = pd.Series(docs)
df_stat.head()#token화

,학수번호,과목소개
0,데이 K0040,인터넷프로그래밍 네트워크 프로그래밍 필요 기법들 이해 클라이언트 서버 통신 프로그램...
1,데이 K0047,"선형모형입문 선형모형 확장 선형모형 기본이론 실습 병행 학습 나 중회귀모형, 분산분..."
2,데이 K0056,정보시스템 보안 관리 & 정보시스템 보안 중요성 관리 방법 학습 나 정보시스템 보안...
3,데이 K0045,정보시스템활용 정보시스템 활용 능력 나 정보시스템 활용 능력 향상 사례분석 세미나 ...
4,데이 K0052,생명과학자료분석 의학 유전자 등 생명과학자료 특성 이 관련 통계분석방법 학습 실제 ...


In [8]:
df_ncs = pd.read_excel('j_ncs.xlsx',encoding='cp949')
df_ncs.head()

,jobname,capability
0,3D 프린터 개발,시장 조사하기 법규 검토하기 기술방식 비교하기 기술방식 선정하기 원가 분석하기 개발...
1,3D 프린터용 제품 제작,시장동향 분석하기 수요조사하기 사업성 검토하기 기획ㅈ품 설정하기 디자인 결정하기 출...
2,CAM,작업계획 수립하기 공정 · 작업내용 계획하기 장비 · 인원 배분하기 도면 결정하기 ...
3,CO2용접,용접작업 안전수칙 파악하기 용접작업장 주변 정리 상태 점검하기 용접 안전보호구 점검...
4,DB엔지니어링,데이터 요구사항 수집하기 데이터 요구사항 분석하기 데이터 요구사항 정의하기 데이터 ...


In [9]:
df_ncs

,jobname,capability
0,3D 프린터 개발,시장 조사하기 법규 검토하기 기술방식 비교하기 기술방식 선정하기 원가 분석하기 개발...
1,3D 프린터용 제품 제작,시장동향 분석하기 수요조사하기 사업성 검토하기 기획ㅈ품 설정하기 디자인 결정하기 출...
2,CAM,작업계획 수립하기 공정 · 작업내용 계획하기 장비 · 인원 배분하기 도면 결정하기 ...
3,CO2용접,용접작업 안전수칙 파악하기 용접작업장 주변 정리 상태 점검하기 용접 안전보호구 점검...
4,DB엔지니어링,데이터 요구사항 수집하기 데이터 요구사항 분석하기 데이터 요구사항 정의하기 데이터 ...
...,...,...
889,환경조사분석,조사요구사항 파악하기 조사관련 법규 파악하기 조사범위 설정하기 조사 수행계획 수립하...
890,환경컨설팅,사업수요 조사하기 사업타당성 평가하기 사업계획서 작성하기 컨설팅 요구분석 하기 프로...
891,회계·감사,회계상 거래 인식하기 전표 작성하기 증빙서류 관리하기 현금시재 관리하기 예금 관리하...
892,회의기획,유치 대상 회의 파악하기 유치 대상 회의 확정하기 유치 대상 회의 관련정보 수집하기...


In [10]:
df_ncs.isnull()

,jobname,capability
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
889,False,False
890,False,False
891,False,False
892,False,False


In [11]:
df_ncs = df_ncs.dropna(axis=0, how='any')

In [12]:
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [13]:
courses_contents = [] 
def append_courses(df_name): #df이름을 매개변수로 
    courses = input('들은 과목들의 학수번호 (쉼표 기준):').split(',')
    return     df_name[df_name['학수번호'].isin(courses)]['과목소개'].sum()


In [14]:
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
    
courses_contents = [] 
def append_courses(df_name): #과목df이름을 매개변수로 
    courses = input('들은 과목들의 학수번호 (쉼표 기준):').split(',')
    return     df_name[df_name['학수번호'].isin(courses)]['과목소개'].sum()

    
def cv_recommend_ten_job():
    cv = CountVectorizer()
    distance = []
    courses = append_courses(df_stat)
    for k in range(0,df_ncs.shape[0]):
        X = np.array(cv.fit_transform([courses, df_ncs.iloc[k,1]]).todense())
        distance.append(cos_sim(X[0], X[1]))
    df = pd.DataFrame(distance, index=df_ncs['jobname'],columns=['recommended job'])
    df = df.sort_values(by=['recommended job'], axis=0,ascending=False)
    return df[:10]

In [23]:
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
    
courses_contents = [] 
def append_course(): 
    courses = input('들은 과목들의 학수번호 (쉼표 기준):').split(',')
    for r in courses:
        index = df_stat.index[df_stat['학수번호'] == r ].tolist()  #학수번호 인덱스뽑기
        #print(index)
        k = df_stat[df_stat['학수번호'] == r ]['과목소개'][index[0]]  #과목소개 추출
        #print(k)
        courses_contents.append(k)  #course_contents리스트에 에펜드
        #print(courses_contents)
    return ''.join(courses_contents)

    
def tfidf_recommend_ten_job():
    cv = TfidfVectorizer()
    distance = []
    courses = append_courses(df_stat)
    #print(courses)
    for k in range(0,df_ncs.shape[0]):
        X = np.array(cv.fit_transform([courses, df_ncs.iloc[k,1]]).todense())
        distance.append(cos_sim(X[0], X[1]))
    df = pd.DataFrame(distance, index=df_ncs['jobname'],columns=['recommended job'])
    df = df.sort_values(by=['recommended job'], axis=0,ascending=False)
    return df

In [61]:
cv_recommend_ten_job()

들은 과목들의 학수번호 (쉼표 기준):경영K0034,경영K0033,경영K0032,경영K0006


,recommended job
jobname,
기업재난관리,0.065419
회계·감사,0.054433
사료생산,0.049639
철도관제,0.045991
일반영업,0.039030
스포츠에이전트,0.038196
이러닝시스템개발,0.037627
수질환경관리,0.035687
특허엔지니어링,0.034700


In [25]:
database = tfidf_recommend_ten_job()

들은 과목들의 학수번호 (쉼표 기준):데이 K0014


In [47]:
database[:20]

,recommended job
jobname,
DB엔지니어링,0.122544
정보시스템운영,0.088071
가상훈련구동엔지니어링,0.085956
제품디자인,0.082640
보안엔지니어링,0.064218
항공기기체설계,0.055686
통신판매,0.054409
가상현실콘텐츠제작,0.051167
색채디자인,0.047591


In [44]:
database[database['recommended job'] != 0]

,recommended job
jobname,
DB엔지니어링,0.122544
정보시스템운영,0.088071
가상훈련구동엔지니어링,0.085956
제품디자인,0.082640
보안엔지니어링,0.064218
...,...
크루즈운영관리,0.001285
화장품제조,0.001033
소방시설설계·감리,0.000994


# 유클리디안

In [55]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))
courses_contents = [] 
def append_course(): 
    courses = input('들은 과목들의 학수번호 (쉼표 기준):').split(',')
    for r in courses:
        index = df_stat.index[df_stat['학수번호'] == r ].tolist()  #학수번호 인덱스뽑기
        #print(index)
        k = df_stat[df_stat['학수번호'] == r ]['과목소개'][index[0]]  #과목소개 추출
        #print(k)
        courses_contents.append(k)  #course_contents리스트에 에펜드
        #print(courses_contents)
    return ''.join(courses_contents)

    
def u_tfidf_recommend_ten_job():
    cv = TfidfVectorizer()
    distance = []
    courses = append_courses(df_stat)
    #print(courses)
    for k in range(0,df_ncs.shape[0]):
        X = np.array(cv.fit_transform([courses, df_ncs.iloc[k,1]]).todense())
        distance.append(euclidean_distance(X[0], X[1]))
    df = pd.DataFrame(distance, index=df_ncs['jobname'],columns=['recommended job'])
    df = df.sort_values(by=['recommended job'], axis=0,ascending=False)
    return df

In [58]:
u_tfidf_recommend_ten_job()

들은 과목들의 학수번호 (쉼표 기준):데이 B0002,데이 B0003,데이 K0004,데이 K0021,데이 K0024,데이 K0014,데이 K0058,경영B0005


,recommended job
jobname,
수력발전설비운영,1.414214
철근콘크리트시공,1.414214
사가공,1.414214
조적미장시공,1.414214
가죽·모피생산,1.414214
...,...
스포츠정보관리,1.372573
해양환경영향평가,1.359565
소음진동측정·분석평가,1.357908
